# Walk through the basic functionality of xgems

**Authors: G. Dan Miron**

For more details, consult the xGEMS [documentation](https://xgems.readthedocs.io/en/latest/) or [repository](https://github.com/gemshub/xgems).


## Importing python packages 

In [ ]:
# comments in python use #
import xgems as xg

## Initalize a chemical engine

- show the `Tab button` functionality (selected function and press enter)
- show help help(xg.ChemicalEngine), and link to the documentation - learn how to initalize an engine object
- initialize with exported GEMS3K files 
- assign the chemical engine to a variable `xgEngine` you can also call it gems, or other name as you please, this is how a standalone gems calculator (having the PC chemical system structure) is initalized

In [ ]:
# initialize with exported GEMS3K files 
# A variable is like a labeled box where you can store information or values.  
# You can put something in the box, change what's inside, and use it later whenever you need it.
# variables can store, numbers, lists, and complex 'objects'
xgEngine = xg.ChemicalEngine('cement-systems/leaching/gems_files/PC_2025-dat.lst')

## Equilibrate, return codes, output
- most used functions reequilibrate, equilibrate
- xgems calculation modes
  
        warmstart - SIA smart initial approximation (using previous results as start guess)
        coldstart - automatic initial approximation (AIA) 
  
- xgems return code
  
        0 "No GEM re-calculation needed"
        1 "Need GEM calculation with LPP (automatic) initial approximation (AIA)"
        2 "OK after GEM calculation with LPP AIA"
        3 "Bad (not fully trustful) result after GEM calculation with LPP AIA"
        4 "Failure (no result) in GEM calculation with LPP AIA"
        5 "Need GEM calculation with no-LPP (smart) IA, SIA using the previous speciation (full DATABR lists only)"
        6 "OK after GEM calculation with SIA"
        7 "Bad (not fully trustful) result after GEM calculation with SIA"
        8 "Failure (no result) in GEM calculation with SIA"
        9 "Terminal error has occurred in GEMS3K (e.g. memory corruption). Restart is required."
- print of chemical system results
- print to txt, ask copilot, chatgpt, etc 


In [ ]:
code = xgEngine.reequilibrate() # reequilibrate using warmstart
#xgEngine.setColdStart()  # once is set is set

In [ ]:
print('GEMS return code', code)

In [ ]:
#print(xgEngine)
with open("result.txt", "w") as f:
    f.write(str(xgEngine))

## How to access the results through functions

- pH, ionicStrength, systemMass

In [ ]:
print(xgEngine.pH())
print(xgEngine.ionicStrength())
print(xgEngine.systemMass(), ' kg')

## How to get composition and properties of phases, species

### A. Get data indexes 

- xgems mainly works with data tables, where the indexes of elements, species, phases are needed - first thing to do is get these indexes by name (as in GEM-Selektor);
- get index of elements Ca, Si
- get index of species H+, Ca+2
- get index of phases aq_gen, Portlandite, Calcite  

In [ ]:
ndx_Ca = xgEngine.indexElement('Ca')
ndx_Si = xgEngine.indexElement('Si')
ndx_Hplus = xgEngine.indexSpecies('H+')
ndx_Caplus2 = xgEngine.indexSpecies('Ca+2')
ndx_aq_gen = xgEngine.indexPhase('aq_gen')
ndx_portlandite = xgEngine.indexPhase('Portlandite')
ndx_calcite = xgEngine.indexPhase('Calcite')

### B. Get properties

- composition of phases solution aq_gen, do help(xgEngine.elementAmountsInPhase), print composition
- what about concentrations of Ca in aq_gen?

In [ ]:
#help(xgEngine.elementAmountsInPhase)

aq_composition = xgEngine.elementAmountsInPhase(ndx_aq_gen)
print(aq_composition)

# this is an array, list in python [ elements, ], elements in the list are accessed by their index, the order in the list is the order of elements in gems chemical system 
# O has the same index in all composition elements lists that xgems returns

In [ ]:
# concentration of Ca in aq_gen mol/L
# 1st we need the phase volume, divide by 1000 from m3 to dm3
aq_volume = xgEngine.phaseVolume(ndx_aq_gen)*1000

# 2nd we get the Ca in aq_gen using its index and divide it by solution volume
# lists in python use square brackets 
print('the concentration of Ca in solution is', aq_composition[ndx_Ca]/aq_volume, 'mol/L')

# in log10 using numpy
import numpy as np
print('the log10 concentration of Ca in solution is', np.log10(aq_composition[ndx_Ca]/aq_volume), 'mol/L')

### How to get Ca/Si in C-S-H?

In [ ]:
## exercise or live depending on the time 

### Get more properties, etc

- saturation index calcite, portlandite
- pH from -log activity of H+

In [ ]:
xgEngine.phaseSatIndex(ndx_calcite)
#xgEngine.phaseSatIndex(ndx_portlandite)

In [ ]:
print('pH from H+ acitivty',-xgEngine.lnActivities()[ndx_Hplus]/np.log(10))

## How to to equilibrate a new composition

- show b, the basic input in gem, follows the index of elements, same order as in GEMS
- create a b_calcite, in moles (needs ndx of O, C)
- get molar mass of calcite, annd use it to convert the b_calcite for 1 kg of calcite 
- add CaCO3 5 g equilibrate, print results
- supress calcite and get sat index, use help(xgEngine.setSpeciesUpperLimit)

In [ ]:
b_PC = xgEngine.elementAmounts().copy() # this is the recipe of our exported system
b_PC

In [ ]:
# 1st we need an empty b for calcite (material), like we would create a compos, 
# but this time we need to do the work, in the future update one would be able to add complex chemical copounds in different units
# and xgems will do the conversions to b as the compos in GEM-Selektor

b_calcite =  xgEngine.elementAmounts().copy()
b_calcite[:] = 1e-9 # empty 

ndx_C = xgEngine.indexElement('C')
ndx_O = xgEngine.indexElement('O')

b_calcite[ndx_Ca] = 1
b_calcite[ndx_C] = 1
b_calcite[ndx_O] = 3
b_calcite

ndx_cal = xgEngine.indexSpecies('Cal')

M_calcite = xgEngine.speciesMolarMasses()[ndx_cal]*1000
M_calcite

In [ ]:
# convert mol for 1 kg of calcite
b_calcite = (1000*b_calcite)/M_calcite

In [ ]:
b_calcite

### the equilibrate function
- this function needs the following variables as input: T (K), P (Pa) and b (mol)

In [ ]:
# add 5 g of calcite and equilibrate
P=1e5 # 10 bar = 1e5 Pa
T=20.0+273.15 # Temperatur in Kelvin

b_recipe_new = b_PC + 0.005*b_calcite

code = xgEngine.equilibrate(T, P, b_recipe_new)
code

In [ ]:
#print(xgEngine)

In [ ]:
# supress species in phase, in GEMS is Upper_KC
xgEngine.setSpeciesUpperLimit('Cal', 0.0)

# equilibrate
code = xgEngine.equilibrate(T, P, b_recipe_new)
code

In [ ]:
#print(xgEngine)

In [ ]:
# retrieve saturation index 
xgEngine.phaseSatIndex(ndx_calcite)

In [ ]:
# thermodynamic properties